# [AHA! Activity Health Analytics](http://casas.wsu.edu/)
[Center for Advanced Studies of Adaptive Systems (CASAS)](http://casas.wsu.edu/)

[Washington State University](https://wsu.edu)
# L2 Data Cleaning: Part 2

## Learner Objectives
At the conclusion of this lesson, participants should have an understanding of:
* Data cleaning 
* Real-world applications of data cleaning

## Acknowledgments
Content used in this lesson is based upon information in the following sources:
* [Pandas website](http://pandas.pydata.org/)

## Data Cleaning Continued
We are going to continue working with the [pd_hoa_activities.csv](https://raw.githubusercontent.com/gsprint23/aha/master/lessons/files/pd_hoa_activities.csv) dataset. Let's load the data into a `pandas` `DataFrame` object. 

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import pandas as pd
import numpy as np

fname = r"files\pd_hoa_activities.csv"
df = pd.read_csv(fname, header=0)

### Missing Data
Let's replace the "?" with `NaN` values so we can more easily detect the fields with missing data:

In [3]:
df.replace("?", np.NaN, inplace=True)

Now let's look at the data in each column.

In [4]:
for col in df.columns:
    ser = df[col]
    print(ser.value_counts())
    print("Number of NaN:", ser.isnull().sum())
    print("***********************************************************************\n")

74    9
18    9
20    9
21    9
22    9
23    9
24    9
25    9
26    9
27    9
28    9
29    9
30    9
31    9
32    9
33    9
34    9
19    9
17    9
36    9
16    9
1     9
2     9
3     9
4     9
5     9
6     9
7     9
8     9
9     9
     ..
63    9
64    9
65    9
66    9
67    9
68    9
69    9
70    9
71    9
72    9
57    9
55    9
38    9
54    9
39    9
40    9
41    9
42    9
43    9
44    9
45    9
46    9
47    9
48    9
49    9
50    9
51    9
52    9
53    9
0     9
Name: pid, dtype: int64
Number of NaN: 0
***********************************************************************

dot    75
2      75
1      75
8      75
4      75
5      75
7      75
6      75
3      75
Name: task, dtype: int64
Number of NaN: 0
***********************************************************************

9       29
10      20
11      18
8       17
7       12
13      10
6        8
12       7
5        7
42       6
52       5
15       5
205      5
222      5
214      5
201      5
34       4
65    

Based on our exploration of the data, we know there are 10 null values in the duration column that we need handle. There are a few ways we do this:
1. Remove the row and/or participant's data with missing information
1. Fill the missing values. One way to do this is by filling each missing value with the average of "similar" instances (e.g. same task, same class, similar age).
1. Leave it alone for now. Handle it on a case by case basis in the later stages of the data analysis pipeline.

We are going to remove the rows with missing information.

In [5]:
print("Before cleaning:", df.shape)  
df.dropna(inplace=True)
index = np.arange(0, len(df))
df.set_index(index, inplace=True)
print("After cleaning:", df.shape)  

Before cleaning: (675, 5)
After cleaning: (665, 5)


### Decode Task
Now, let's decode the integer values associated with the task column by replacing them with a more human-readable text label. We will use a dictionary to story the integer to string mappings for task codes and replace the integers with the strings in place.

In [6]:
task_decoder = {"1": "Water Plants", "2": "Fill Medication Dispenser", "3": "Wash Countertop", \
               "4": "Sweep and Dust", "5": "Cook", "6": "Wash Hands", "7": "Perform TUG", \
               "8": "Perform TUG w/Questions", "dot": "Day Out Task"}

def decode_task(df):
    '''
    
    '''
    ser = df["task"]
    for key in task_decoder:
        ser.replace(key, task_decoder[key], inplace=True)
decode_task(df)
print(df.head(n=11))

    pid                       task duration  age class
0     0               Water Plants      146   72   HOA
1     0  Fill Medication Dispenser      210   72   HOA
2     0            Wash Countertop      241   72   HOA
3     0             Sweep and Dust      328   72   HOA
4     0                       Cook      229   72   HOA
5     0                 Wash Hands       38   72   HOA
6     0                Perform TUG       10   72   HOA
7     0    Perform TUG w/Questions       10   72   HOA
8     0               Day Out Task      680   72   HOA
9     1               Water Plants       63   54   hoa
10    1  Fill Medication Dispenser      202   54   hoa


Looking at our data frame now, we see that the task category is much more readable. This will be especially useful for generating plots with task labels.

### Check Numeric Data Types
Now, let's check out the data types for our two numeric columns, duration and age:

In [7]:
print(df["duration"].dtype)
print(df["age"].dtype)

object
int64


We see that the age column is an integer type, but the duration column is an object type. This means that Pandas was unable to infer this column contained all integers when it was read in, which makes sense since we know there were "?"s in the duration column. Since we have replaced the "?" with `NaN`, let's convert it to integer now:

In [8]:
df["duration"] = df["duration"].astype(np.int)
print(df["duration"].dtype)

int32


### Clean Class
Lastly, we are going to clean the class column. This column is quite messy compared to the other columns we have worked with. We will use a simple rule based system to handle the various spellings and word choices that represent "HOA" and "PD".

Note: If there are entries that we cannot classify as one of the above labels, we will overwrite the entry with a null value (`NaN`) to represent missing data.

In [9]:
def clean_class(df):
    '''
    
    '''
    ser = df["class"].copy()
    
    for i in range(0, len(ser), 1):
        curr = str(ser[i])
        curr = curr.lower()
        if "hoa" in curr or "healthy" in curr:
            ser[i] = "HOA"
        elif "pd" in curr or "parkinson" in curr:
            ser[i] = "PD"
        else:
            print("Unrecognized status: %d, %s" %(i, ser[i]))
            ser[i] = np.NaN
        
    df["class"] = ser

clean_class(df)
print(df.head())
print(df["class"].value_counts())

   pid                       task  duration  age class
0    0               Water Plants       146   72   HOA
1    0  Fill Medication Dispenser       210   72   HOA
2    0            Wash Countertop       241   72   HOA
3    0             Sweep and Dust       328   72   HOA
4    0                       Cook       229   72   HOA
HOA    446
PD     219
Name: class, dtype: int64


Now, we will write the cleaned data frame out to a new file. Our dataset is now cleaned and ready for use in the next step of our data analysis pipeline. Depending on what we want to do with the data, this could be continuing exploration by generating visualizations of the data, or perhaps scaling the features in preparation for machine learning.

In [9]:
out_fname = r"files\pd_hoa_activities_cleaned.csv"
df.to_csv(out_fname, index=False) # don't write out the index column